In [3]:
import polars as pl

# --- 1. CHARGER LES "MASTER FILES" (en mode "scan" rapide) ---
try:
    df_sirene = pl.scan_parquet("../Data/processed/sirene_infos.parquet").select(
        "siren", "dateCreationUniteLegale"
    )
    # ▼▼▼ CORRECTION ICI ▼▼▼
    df_bilan = pl.scan_parquet("../Data/processed/sirene_bilan.parquet").select(
        "siren", "AnneeClotureExercice" 
    )
    print("Fichiers 'master' trouvés.")
except Exception as e:
    print(f"ERREUR: Fichiers 'processed' non trouvés. As-tu lancé 'make process'?")
    print(e)

# --- 2. DÉFINIR LA COHORTE (Notre population cible) ---
print("Filtrage: Cohorte 2018 (SIRENE)...")
df_demo_2018 = df_sirene.filter(
    pl.col("dateCreationUniteLegale").dt.year() == 2018
)

# On compte combien d'entreprises ont été créées en 2018
total_cohort_2018 = df_demo_2018.collect().height
print(f"Entreprises créées en 2018 (Cohorte) : {total_cohort_2018}")

# --- 3. DÉFINIR LES BILANS (Nos features expertes) ---
print("Filtrage: Bilans 2019 (INPI)...")
# ▼▼▼ CORRECTION ICI ▼▼▼
df_bilan_2019 = df_bilan.filter(
    pl.col("AnneeClotureExercice") == 2019
)

# --- 4. LE "TEST DE JOINTURE" (L'étape critique) ---
print("Test de la jointure (inner join)...")

# On fait une jointure "inner" pour ne garder QUE les lignes communes
df_overlap = df_demo_2018.join(
    df_bilan_2019, 
    on="siren", 
    how="inner"
)

# On compte le résultat
total_overlap = df_overlap.collect().height

# --- 5. LE VERDICT ---
print("---")
print("--- VERDICT DU DIAGNOSTIC ---")
print(f"Entreprises totales créées en 2018 : {total_cohort_2018}")
print(f"Entreprises de 2018 AVEC un bilan en 2019 : {total_overlap}")

if total_cohort_2018 > 0:
    coverage_percent = (total_overlap / total_cohort_2018) * 100
    print(f"Taux de couverture des données : {coverage_percent:.2f}%")
else:
    print("Aucune entreprise trouvée pour 2018.")

print("---")

if total_overlap < 1000:
    print("ATTENTION: Très peu de données communes. Le modèle risque d'être faible.")
elif total_overlap < 5000:
    print("OK: C'est assez pour un PoC (Proof of Concept), mais c'est peu.")
else:
    print("SUPER: Nous avons des milliers de lignes communes. C'est parfait pour le ML !")

Fichiers 'master' trouvés.
Filtrage: Cohorte 2018 (SIRENE)...
Entreprises créées en 2018 (Cohorte) : 845851
Filtrage: Bilans 2019 (INPI)...
Test de la jointure (inner join)...
---
--- VERDICT DU DIAGNOSTIC ---
Entreprises totales créées en 2018 : 845851
Entreprises de 2018 AVEC un bilan en 2019 : 26361
Taux de couverture des données : 3.12%
---
SUPER: Nous avons des milliers de lignes communes. C'est parfait pour le ML !


In [6]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- 1. CHARGER LES "MASTER FILES" ---
PATH_SIRENE = "../Data/processed/sirene_infos.parquet"
PATH_BILAN = "../Data/processed/sirene_bilan.parquet"

if not os.path.exists(PATH_SIRENE) or not os.path.exists(PATH_BILAN):
    print("ERREUR: Fichiers 'processed' non trouvés. Lance 'make process' d'abord.")
else:
    df_sirene = pl.read_parquet(PATH_SIRENE)
    df_bilan = pl.read_parquet(PATH_BILAN)
    print(f"Fichier SIRENE (infos) chargé: {df_sirene.shape}")
    print(f"Fichier INPI (bilans) chargé: {df_bilan.shape}")

# --- 2. DÉFINIR LA COHORTE (Itération 1) ---
print("Filtrage: Cohorte 2018 & Bilans 2019...")

# A. Cohorte 2018 (SIRENE)
# Ce sont nos 845k entreprises
df_demo = df_sirene.filter(
    pl.col("dateCreationUniteLegale").dt.year() == 2018
)

# B. Bilans 2019 (INPI)
df_bilan_2019 = df_bilan.filter(
    pl.col("AnneeClotureExercice") == 2019
)

# --- 3. LE GRAND MARIAGE (LEFT JOIN) ---
# On garde toutes les entreprises de 2018 (gauche)
# On attache les bilans 2019 (droite) s'ils existent
df_final_ml = df_demo.join(
    df_bilan_2019, 
    on="siren", 
    how="left"
)
print(f"Shape après jointure: {df_final_ml.shape}") # Doit être (845851, ...)

# --- 4. CRÉATION DE LA TARGET (Y) ---
print("Création de la Cible (is_failed_in_3y)...")

df_final_ml = df_final_ml.with_columns(
    # Date limite = 3 ans après la création
    # ▼▼▼ CORRECTION ICI ▼▼▼
    (pl.col("dateCreationUniteLegale").dt.offset_by("3y")).alias("date_limite_3_ans")
    # ▲▲▲ CORRECTION ICI ▲▲▲
).with_columns(
    # 1 = Faillite si Fermé AVANT la date limite
    pl.when(
        (pl.col("dateFermeture").is_not_null()) & # dateFermeture n'est pas null
        (pl.col("dateFermeture") < pl.col("date_limite_3_ans"))
    ).then(1)
    .otherwise(0)
    .alias("is_failed_in_3y")
)

# --- 5. NETTOYAGE FINAL (TRÈS IMPORTANT) ---
print("Nettoyage des 'null' (entreprises sans bilan)...")
# Remplir les 'null' des entreprises sans bilan par 0
cols_financieres = [col for col in df_bilan.columns if col not in ['siren', 'DateClotureExercice', 'AnneeClotureExercice']]
df_final_ml = df_final_ml.with_columns(
    pl.col(cols_financieres).fill_null(0)
)

print("---")
print("DATASET ML PRÊT !")
print(f"Shape finale : {df_final_ml.shape}")
print(df_final_ml.head())

Fichier SIRENE (infos) chargé: (28882409, 7)
Fichier INPI (bilans) chargé: (3706645, 21)
Filtrage: Cohorte 2018 & Bilans 2019...
Shape après jointure: (845890, 27)
Création de la Cible (is_failed_in_3y)...
Nettoyage des 'null' (entreprises sans bilan)...
---
DATASET ML PRÊT !
Shape finale : (845890, 29)
shape: (5, 29)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ siren     ┆ dateCreat ┆ dateFerme ┆ categorie ┆ … ┆ ratio_res ┆ ratio_res ┆ date_limi ┆ is_faile │
│ ---       ┆ ionUniteL ┆ ture      ┆ Juridique ┆   ┆ ultat_fin ┆ ultat_exc ┆ te_3_ans  ┆ d_in_3y  │
│ str       ┆ egale     ┆ ---       ┆ UniteLega ┆   ┆ ancier    ┆ eptionnel ┆ ---       ┆ ---      │
│           ┆ ---       ┆ date      ┆ le        ┆   ┆ ---       ┆ ---       ┆ date      ┆ i32      │
│           ┆ date      ┆           ┆ ---       ┆   ┆ f64       ┆ f64       ┆           ┆          │
│           ┆           ┆           ┆ i64       ┆   ┆           ┆         

Utiliser ML FLOW 